In [ ]:
!pip install openai

**<h1>AI POWERED CHATBOT</h1>**

In [ ]:
import requests
import getpass

api_key = getpass.getpass("Enter your OpenRouter API key: ")
API_URL = "https://openrouter.ai/api/v1/chat/completions"

# ✅ Switch to a lower-cost model
MODEL_NAME = "mistralai/mistral-small"  # Use Mistral Small for lower token usage

def chat_with_openrouter():
    print("Chatbot: Hello! Type 'exit' to end the conversation.")

    conversation = []

    while True:
        user_input = input("You: ")
        if user_input.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        # ✅ Keep only the last 5 messages to save tokens
        if len(conversation) > 5:
            conversation = conversation[-5:]

        conversation.append({"role": "user", "content": user_input})

        payload = {
            "model": MODEL_NAME,
            "messages": conversation
        }

        headers = {
            "Authorization": f"Bearer {api_key}",
            "Content-Type": "application/json"
        }

        response = requests.post(API_URL, json=payload, headers=headers)

        try:
            response_json = response.json()

            if response.status_code == 200 and "choices" in response_json:
                reply = response_json["choices"][0]["message"]["content"]
                print(f"Chatbot: {reply}")
                conversation.append({"role": "assistant", "content": reply})
            else:
                print(f"API Error: {response_json}")  # Print error response

        except Exception as e:
            print(f"Unexpected Error: {e}")

# Run the chatbot
chat_with_openrouter()


Enter your OpenRouter API key: ··········
Chatbot: Hello! Type 'exit' to end the conversation.
You: hi
Chatbot: Hello! How can I assist you today?
You: what is your name ?
Chatbot: I don't have a name, but you can give me one if you'd like! What would you like to call me?
You: exit
Chatbot: Goodbye!


**<h1>AI RESUME ANALYSER</h1>**

In [ ]:
!pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 46.3 MB/s eta 0:00:00


In [ ]:
import openai
import requests
import fitz  # PyMuPDF for PDF reading
import getpass

# 🔑 Get API Key securely
api_key = getpass.getpass("Enter your OpenRouter API key: ")

# 🌐 Choose API Provider ("openrouter" or "openai")
API_PROVIDER = "openrouter"

# 🧠 Choose a Model (Use a cheaper one if limited tokens)
MODEL_NAME = "mistralai/mistral-7b-instruct"  # Use a low-cost model
  # OpenRouter Model
# MODEL_NAME = "gpt-4-turbo"  # OpenAI Model (if using OpenAI API)

# 📌 Extract text from PDF
def extract_text_from_pdf(pdf_path):
    doc = fitz.open(pdf_path)
    text = ""
    for page in doc:
        text += page.get_text("text") + "\n"
    return text

# 📌 Send resume text to LLM for evaluation
def analyze_resume(resume_text):
    prompt = f"""
    You are an AI Resume Expert. Analyze the resume below and provide feedback on:
    1. Overall quality (clarity, structure, formatting)
    2. Skills alignment with Data Science/AI jobs
    3. Areas for improvement (missing details, weak points)

    Resume Text:
    {resume_text}

    Provide your response in clear bullet points.
    """

    payload = {
        "model": MODEL_NAME,
        "messages": [{"role": "user", "content": prompt}]
    }

    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }

    # 🌐 API Request (Choose OpenRouter or OpenAI)
    if API_PROVIDER == "openrouter":
        response = requests.post("https://openrouter.ai/api/v1/chat/completions", json=payload, headers=headers)
    else:
        openai.api_key = api_key
        response = openai.ChatCompletion.create(**payload)

    # 🔄 Handle response
    try:
        response_json = response.json() if API_PROVIDER == "openrouter" else response
        if "choices" in response_json:
            return response_json["choices"][0]["message"]["content"]
        else:
            return f"API Error: {response_json}"
    except Exception as e:
        return f"Error processing response: {e}"

# 📌 Main function to run the analyzer
def main():
    pdf_path = input("Enter the path to your resume PDF file: ")

    print("\n🔍 Extracting text from resume...\n")
    resume_text = extract_text_from_pdf(pdf_path)

    print("🧠 Analyzing resume...\n")
    feedback = analyze_resume(resume_text)

    print("\n📋 Resume Analysis Report:")
    print(feedback)

# 🚀 Run the program
main()


Enter your OpenRouter API key: ··········
Enter the path to your resume PDF file: /content/Engineering_Industrial_Tech_Resume.pdf

🔍 Extracting text from resume...

🧠 Analyzing resume...


📋 Resume Analysis Report:
 1. Overall Quality: The resume is well-organized, clear, and easy to read. It has a clear structure with headings and bullet points, which make it easy for the reader to quickly understand the candidate's qualifications. The formatting is also good, with appropriate use of whitespace and bolding to separate sections. However, there could be more conciseness in explaining responsibilities and tasks, and the resume could benefit from a summarized snapshot of professional experience at the top.

2. Skills Alignment with Data Science/AI Jobs: While the candidate has good technical skills in modeling and drafting software, data science and AI jobs typically require proficiency in programming languages such as Python, R, SQL, and Java. There's no mention of any programming or mac

**<h1>NEWS SUMMARIZER</h1>**

In [ ]:
import openai
import requests
import panel as pn
from bs4 import BeautifulSoup

# Set up Panel extension
pn.extension()

# 🔑 Input for API Key
api_key = pn.widgets.PasswordInput(name="Enter OpenRouter API Key")

# 🌐 Choose API Provider
api_provider = pn.widgets.Select(name="API Provider", options=["openrouter", "openai"], value="openrouter")

# 🧠 Choose a Model
model_name = pn.widgets.Select(name="Model", options=["mistralai/mistral-7b-instruct", "gpt-4-turbo"], value="mistralai/mistral-7b-instruct")

# 📌 Function to extract text from a URL
def extract_text_from_url(url):
    try:
        response = requests.get(url)
        soup = BeautifulSoup(response.text, "html.parser")
        paragraphs = soup.find_all("p")
        article_text = "\n".join([p.get_text() for p in paragraphs])
        return article_text[:5000]  # Limit tokens
    except Exception as e:
        return f"Error extracting text: {e}"

# 📌 Function to summarize text using LLM
def summarize_news(article_text):
    prompt = f"""
    Summarize the following news article concisely:

    Article:
    {article_text}

    Summary:
    """

    payload = {
        "model": model_name.value,
        "messages": [{"role": "user", "content": prompt}]
    }

    headers = {
        "Authorization": f"Bearer {api_key.value}",
        "Content-Type": "application/json"
    }

    if api_provider.value == "openrouter":
        response = requests.post("https://openrouter.ai/api/v1/chat/completions", json=payload, headers=headers)
    else:
        openai.api_key = api_key.value
        response = openai.ChatCompletion.create(**payload)

    try:
        response_json = response.json() if api_provider.value == "openrouter" else response
        if "choices" in response_json:
            return response_json["choices"][0]["message"]["content"]
        else:
            return f"API Error: {response_json}"
    except Exception as e:
        return f"Error processing response: {e}"

# 📌 Panel UI Components
url_input = pn.widgets.TextInput(name="News Article URL")
text_input = pn.widgets.TextAreaInput(name="Or Paste News Article Text", height=200)
summary_output = pn.pane.Markdown("", width=600)

def process_summary(event):
    if url_input.value:
        article_text = extract_text_from_url(url_input.value)
    else:
        article_text = text_input.value

    summary = summarize_news(article_text)
    summary_output.object = f"## News Summary:\n{summary}"

summarize_button = pn.widgets.Button(name="Summarize", button_type="primary")
summarize_button.on_click(process_summary)

# 📌 Layout
app = pn.Column(
    "# 📰 News Summarizer",
    api_key,
    api_provider,
    model_name,
    url_input,
    "**OR**",
    text_input,
    summarize_button,
    summary_output
)

# Run the Panel App
app.servable()


<ipython-input-3-3138199e6fe9>:7: UserWarning: Using Panel interactively in Colab notebooks requires the jupyter_bokeh package to be installed. Install it with:

    !pip install jupyter_bokeh

and try again.
  pn.extension()


Column
    [0] Markdown(str)
    [1] PasswordInput(name='Enter OpenRouter A...)
    [2] Select(name='API Provider', options=['openrouter', 'openai'], value='openrouter')
    [3] Select(name='Model', options=['mistralai/mistral-7b-ins...], value='mistralai/mistral-7b-inst...)
    [4] TextInput(name='News Article URL')
    [5] Markdown(str)
    [6] TextAreaInput(height=200, name='Or Paste News A...)
    [7] Button(button_type='primary', name='Summarize')
    [8] Markdown(str, width=600)